**0. Importing packages**

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import pickle
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.cuda.amp import autocast, GradScaler
from scipy.fft import fft2, ifft2, fftfreq

**1. Generating datasets**


In [2]:
L = 10 #domain length
N = 100 #number of grid points per dimension
delta_x = L / N
delta_y = L / N
delta_t = 0.000001
iterations = 2000 #number of timesteps per data example

num_datasets = 5
num_datapoints = 1000 # num datapoints per dataset
sequence_length = 10 # num snapshots per datapoint

frequencies = fftfreq(N, d=1/N) #frequencies used in scipy fft
k1 = np.tile(frequencies,(N,1)) #frequencies associated to x variable in fourier series
k2 = k1.T #frequencies associated to y variable in fourier series
spectral_laplace = - (2*np.pi*(1 / L))**2 * (k1**2 + k2**2) #helps calculate laplacian applied to
                                                            #fourier coefficients


In [ ]:
#Generate datasets and save them
for k in range(num_datasets):
    dataset = []
    for i in range(num_datapoints):
        if i%10 == 0:
            print(f'Generating data point {i}')
        #Generate random parameters and initial conditions
        D = random.uniform(1.0, 10.0)
        l = random.uniform(1.0, 10.0)
        c = np.random.uniform(-0.01, 0.01, (N, N))

        #Generate data
        current_simulation = []
        for j in range(iterations):
            #We only add sequence_length number of snapshots to current_simulation
            if j%(iterations / sequence_length) == 0:
                current_simulation.append(c)
            #Compute c at next time step
            numerator = fft2(c) + delta_t*D*spectral_laplace*fft2(c**3-c)
            denominator = 1+delta_t*D*l*spectral_laplace*spectral_laplace
            c = ifft2( numerator / denominator ).real

        #Add new data to dataset
        dataset.append({'D': D, 'l': l, 'data': current_simulation})

    print(f'Done generating dataset {k+1}')
    file_path = f'ch_video_dataset_{k+1}.pkl'

    with open(file_path, "wb") as file:
        pickle.dump(dataset, file)
    print(f'Done with dataset {k+1}!')

**2. Loading and Preprocessing datasets**

In [6]:
#Let's load the dataset
dataset = []
for i in range(num_datasets):
    with open(f'ch_video_dataset_{i+1}.pkl', 'rb') as f:
        dataset = dataset + pickle.load(f)

#Reshaping data into dimensions time_step x nChannels x Height x Width
for i in range(len(dataset)):
    dataset[i]['data'] = np.reshape(np.array(dataset[i]['data']), (sequence_length,1,N,N))

#Calculating mean and std deviation for z-score normalization
sum = 0
for i in range(len(dataset)):
    sum += np.sum(dataset[i]['data'])
mean = sum / (len(dataset) * sequence_length * N * N)

sum = 0
for i in range(len(dataset)):
    sum += np.sum((dataset[i]['data'] - mean)**2)
sigma = np.sqrt(sum / (len(dataset) * sequence_length * N * N))

In [7]:
#Defining dataset class and dataloader objects
class MyCustomDataset(Dataset):
    def __init__(self, original_dataset, transform=None):
        self.original_dataset = original_dataset
        self.transform = transform

    def __len__(self):
        # Return the total number of samples
        return len(self.original_dataset)

    def __getitem__(self, idx):
        # Get one sample from the original dataset
        sample = self.original_dataset[idx]

        # Extract the video data and label data
        video_data = sample['data'] # This should be a NumPy array or PIL Image
        label_data = torch.tensor([sample['D'], sample['l']]).float()

        # Apply transformations to the video (if necessary)
        if self.transform:
            video_data = self.transform(video_data)

        return video_data, label_data


def pre_process(video):
    video = torch.from_numpy(video)
    return ((video - mean) / sigma).float()

train_data = MyCustomDataset(dataset[100:5000], pre_process)
cv_data = MyCustomDataset(dataset[0:100], pre_process)

batch_size = 128
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
cv_loader = DataLoader(cv_data, batch_size=batch_size, shuffle=False)

**3. Defining training and evaluation functions**

In [8]:
#Evaluation function
def evaluate_model(model, val_loader, loss_function, device):
    #Evaluates the model on the validation dataset.
    #Set the model to evaluation mode
    model.eval()

    running_val_loss = 0.0

    #Disable gradient calculations
    #We don't need to calculate gradients for validation, which saves memory and computation.
    with torch.no_grad():
        for inputs, labels in val_loader:
            # Move data to the specified device
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(inputs)

            # Calculate the loss
            loss = loss_function(outputs, labels)

            # Accumulate the loss
            # .item() gets the raw Python number from the tensor
            # inputs.size(0) is the batch size
            running_val_loss += loss.item() * inputs.size(0)

    #Calculate the average loss over the entire validation set
    average_val_loss = running_val_loss / len(val_loader.dataset)

    return average_val_loss

In [9]:
#Training loop
def training_loop(model, train_loader, cv_loader, criterion, optimizer, scheduler, device,
                  num_epochs=50, patience=10):

  #Early stopping parameters
  epochs_no_improve = 0
  min_val_loss = float('inf')
  best_model_state = None

  for epoch in range(num_epochs):
      model.train()
      running_train_loss = 0
      for feature_batch, label_batch in train_loader:
          #Move data to GPU
          feature_batch = feature_batch.to(device)
          label_batch = label_batch.to(device)

          #Zero the parameter gradients
          optimizer.zero_grad()

          #forward + backward + optimize
          with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):
            outputs = model(feature_batch)
            loss= criterion(outputs, label_batch)

          running_train_loss += loss.item() * feature_batch.size(0)

          scaler.scale(loss).backward()
          scaler.step(optimizer)

          scaler.update()
      #Print statistics
      avg_train_loss = running_train_loss / len(train_loader.dataset)
      avg_val_loss = evaluate_model(model, cv_loader, criterion, device)
      scheduler.step(avg_val_loss)
      print(f'Epoch: {epoch}, Avg train loss: {avg_train_loss}, Avg val loss: {avg_val_loss}')

      if avg_val_loss < min_val_loss:
        min_val_loss = avg_val_loss
        epochs_no_improve = 0
        # Save the best model state
        best_model_state = model.state_dict()

      else:
        epochs_no_improve += 1

      if epochs_no_improve == patience:
        print(f'Early stopping triggered after {epoch+1} epochs!')
        break
  print(f'Finished training with min_val_loss of {min_val_loss}')

  return best_model_state

**4. Setting up model and hyperparameters for training**

In [10]:
#Defining Convolutional RNN class
class ConvRNN(nn.Module):
    def __init__(self, device=None):
        super(ConvRNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32, 32, 3, padding = 1)
        self.fc1 = nn.Linear(32 * 25 * 25, 512)
        self.rnn = nn.RNN(512, 512, num_layers=3, nonlinearity='relu', batch_first=True, device=device)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x):
        #Shape of input x is (batch_idx) x (time_step) x (nChannels) x (Height) x (Width)

        #Convolutional step
        b_size, t_size, c_size, h_size, w_size = x.shape
        x = torch.reshape(x, (b_size * t_size, c_size, h_size, w_size))
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))

        x = x.view(b_size, t_size, -1)

        out, _ = self.rnn(x)
        out = self.dropout(out[:,-1,:])
        out = self.fc2(out)
        return out

In [11]:
#Let's try using CUDA
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and being used.")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU.")

#Constructing Gradient Scaler
scaler = GradScaler()

#Constructing model
model = ConvRNN(device)

#Loss function
criterion = nn.MSELoss()

#Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

#Set up lr scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2, patience=5)

model.to(device)

GPU is available and being used.


/tmp/ipython-input-3084056367.py:10: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


ConvRNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=20000, out_features=512, bias=True)
  (rnn): RNN(512, 512, num_layers=3, batch_first=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)

**5. Train model and output best model state**

In [12]:
output_model = training_loop(model, train_loader, cv_loader, criterion, optimizer, scheduler, device,
                  num_epochs=50, patience=20)

/tmp/ipython-input-852657703.py:22: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device.type == 'cuda')):


Epoch: 0, Avg train loss: 12.668449944087437, Avg val loss: 5.630335330963135
Epoch: 1, Avg train loss: 5.32409631651275, Avg val loss: 4.814549922943115
Epoch: 2, Avg train loss: 4.895151945620167, Avg val loss: 4.3019700050354
Epoch: 3, Avg train loss: 4.758860840700111, Avg val loss: 4.6155924797058105
Epoch: 4, Avg train loss: 4.739522348909962, Avg val loss: 4.228938579559326
Epoch: 5, Avg train loss: 4.796003853934152, Avg val loss: 4.107840538024902
Epoch: 6, Avg train loss: 4.584124496226408, Avg val loss: 5.097446918487549
Epoch: 7, Avg train loss: 4.638980679803965, Avg val loss: 4.605401515960693
Epoch: 8, Avg train loss: 4.591412334831394, Avg val loss: 4.129711627960205
Epoch: 9, Avg train loss: 4.428692783822819, Avg val loss: 5.4402079582214355
Epoch: 10, Avg train loss: 4.515507722971391, Avg val loss: 4.554859161376953
Epoch: 11, Avg train loss: 4.683591982004594, Avg val loss: 4.094710826873779
Epoch: 12, Avg train loss: 4.365301838310398, Avg val loss: 3.838188409805